In [161]:
import tkinter 
from tkinter import ttk 
import sqlite3
import pandas as pd
from datetime import date

choosed= 0



def closeFrame(self):
    self.pack_forget()
    
def opendb():
    global con, cur
    con = sqlite3.connect('mainbase.db')
    cur = con.cursor()

    return con,cur

def openFrame(self,sF = 0):
    '''Abre uma nova janela

        @param self: Janela que sera aberta

        @param sF se é um sub Frame (default 0)

    '''    
    
    global nowFrame
    if sF == 0:
        closeFrame(nowFrame)
    self.pack(fill='both', expand= True)
    
    nowFrame = self
    return(nowFrame)

def seachStd(self, typ, datab = 0, filt= 0):
    ''' pesquisa por um dado dentro do banco.
        @param typ: tipo de pesquisa : 0 basica, com saida em toast; 1 complexa, com filtos, saidas no menu de exibição.
        @param self: Entry qual sera buscado. 
        @param datab: base qual sera pesquisada; EX : estudantes, professores, curso
        @param filt: filtro selecionado
    '''
    
    if typ == 0:
        opendb()   
        stdget = self.get()
        cur.execute("SELECT nome FROM estudantes WHERE nome LIKE ?", ('%' + stdget + ' %',))
        founded = cur.fetchone()
        con.close()
        if founded:
            tstMensage(0, "O Aluno foi ENCONTRADO !")
            print(founded)
        else:
            tstMensage(0,'        O Aluno não foi encontrado; \n tente ver se foi escrito corretamente.')
    
                    
    elif typ == 1:
    
        #closeFrame(rsFrame)
        opendb()
        stdget = self.get()
        stdget = ("'%"+stdget+"%'")
        datab= datab.get()
        filt = filt.get()
        query = """
            SELECT * 
            FROM %s 
            WHERE %s LIKE %s
            """ %(datab, filt, stdget)
        df = pd.read_sql_query(query, con)
        con.close()
        
        rsFrame = tkinter.Frame(shwFrame)
        rsFrame.grid(row=4, column=1)

        rowSet= df.to_numpy().tolist()
        rsTree = ttk.Treeview(rsFrame, show='headings', columns=l1,height=3)
        rsTree.grid()

        for i in l1:
            rsTree.column(i, width=90, anchor='c')
            rsTree.heading(i, text= str(i))
        for dt in rowSet:
            v = [r for r in dt]
            rsTree.insert("", 'end', iid= v[0], values= v)      
    else:
        tstMensage(2,'tipo de pesquisa incorreto')
        
def prtList(database):
    
    opendb()
    dbcur = con.cursor()
    dbcur.execute("SELECT * FROM %s;" %(database))
    print('=========================================================')
    for line in dbcur.fetchall():
        print(line)
    print('=========================================================')
    con.close()


def shwsl(datab, typ=0):

    global l1,df
    opendb()
    if typ == 0:
        df = pd.read_sql_query("SELECT * FROM %s ORDER BY id ASC" %(datab),con)
    elif typ == 1:
        df = pd.read_sql_query("""SELECT curso.Nome as 'Nome do Curso', curso.CGhoraria as 'Carga Horaria', turno as Turno, professores.nome as Professor, estudantes.nome  
                                FROM classes
                                LEFT JOIN  estudantes on  classes.estudantesId = estudantes.id 
                                LEFT JOIN  curso on  classes.cursoID = curso.ID
                                LEFT JOIN  professores on  curso.professor = professores.id
                                """ ,con)

    elif typ == 2:
        df = pd.read_sql_query("""SELECT curso.id as ID , curso.Nome as 'Nome do Curso', curso.CGhoraria as 'Carga Horaria', professores.nome as Professor
                                FROM curso 
                                LEFT JOIN  professores on  curso.professor = professores.id
                                ORDER BY id ASC
                                """ ,con)        

    con.close()
    l1= list(df)
    return l1,df

def shwList(self,parent,datab, typ= 0,dbevent = 0, hg= 10):
    ''' exibe em formato de lista a tabela
        @param self: frame pertencente.
        @param parent:frame pai qual pertence
        @param datab: table qual sera utilizada
        @param typ: 0: mostrar tudo, 1: filtro para classes, 2:filtro para cursos
        @param dbevent: evento ao dar double click na tabela; padrao 0 NADA
        @param hg: quantidades de linhas da tabela  
    '''

    
    global shwSFtlist,choosed, l1, df
    
    openFrame(parent)
    shwsl(datab,typ)   

    rowSet= df.to_numpy().tolist()

    

    shwSFtlist = ttk.Treeview(self, show='headings', columns=l1,height= hg)
    shwSFtlist.grid(row= 2, column=1)
    
    if dbevent == 1:
        shwSFtlist.bind("<<TreeviewSelect>>",SlList)    

    for i in l1:
        shwSFtlist.column(i, width=95, anchor='c')
        shwSFtlist.heading(i, text= str(i))
    for dt in rowSet:
        v = [r for r in dt]
        shwSFtlist.insert("", 'end', iid= v[0], values= v)

def upd():
    for i in shwSFtlist.get_children():
        shwSFtlist.delete(i)

def tstMensage(typ, msg= ""):
    ''' exibe um pop up na tela.

        @param typ: qual será o tipo de interface; 0 mensagem, 1 confirmação, 2 erro.
        @param msg: mensagem que sera mostrada 
    '''

    global toast
    if typ == 0:
        toast = tkinter.Tk()
        toast.title('MENSAGEM!')
        toast.geometry('200x70')
        msg = tkinter.Label(toast, text= msg)
        msg.pack(side='top')
        tstButton = tkinter.Button(toast, text='OK', command= lambda:toast.destroy())
        tstButton.pack(side='bottom', pady=10)
        toast.mainloop()
    elif typ == 1:
        global conf
        conf = False
        toast = tkinter.Tk()
        toast.title('CONFIRMAÇÃO!')
        toast.geometry('270x80')
        msg = tkinter.Label(toast, text= msg)
        msg.pack(side='top')
        msg1 = tkinter.Label(toast, text='deseja confirmar ?')
        msg1.pack(side='top')
        tstButton = tkinter.Button(toast, text='cancelar', padx=5, command= lambda:[tstConf(False), toast.destroy()])
        tstButton.pack(side='left', pady=10)
        tstButton1 = tkinter.Button(toast, text='ok', padx=20, command=lambda:[tstConf(True), toast.destroy()])
        tstButton1.pack(side='right', pady=10)
        toast.mainloop()

        return(conf, toast)
    elif typ == 2:
        toast = tkinter.Tk()
        toast.title('OPS!')
        toast.geometry('200x70')
        msg0= tkinter.Label(toast, text='Algo deu errado. ')
        msg0.pack(side='top')
        msg = tkinter.Label(toast, text= 'detalhes :' + msg)
        msg.pack(side='top')
        tstButton = tkinter.Button(toast, text='OK', command= lambda:toast.destroy())
        tstButton.pack(side='bottom', pady=10)

        toast.mainloop()
    elif typ == 3:

        global subf,lbSprof,pmtmask, toastt,lbID,lbMAT,spcg,bxname
        pmtmask = ""
        toast = tkinter.Tk()
        toast.title('Cadastro de Cursos')
        toast.geometry('500x300')

        newId('curso')
        ftMat('curso','C')

        toastt = tkinter.Frame(toast)
        toastt.pack()#grid(row= 0, column=1)
        lbname = tkinter.Label(toastt, text="Nome do Curso: ").grid(row= 1, column=0, pady= 5)
        bxname = tkinter.Entry(toastt, width= 25, borderwidth=1, relief= 'groove' , bg="white")
        bxname.grid(row= 1, column=1, pady= 5)
        lbID = tkinter.Label(toastt, text=" ID: %s" %(nid))
        lbMAT = tkinter.Label(toastt, text="Matricula: %s" %(mmt))
        lbcg = tkinter.Label(toastt, text='Carga Horaria: ').grid(row=2, column=0, pady=5, sticky= 'w')
        spcg = tkinter.Spinbox(toastt, from_=0, to= 10000, increment=10, width=6, relief='groove')
        spcg.grid(row=2, column=1, pady=5, sticky='e')
        btSprof = tkinter.Button(toastt, text='Selecionar Professor',command= lambda:shwList(subf,subf,'professores',0,1)).grid(row=3, column=0)
        lbSprof = tkinter.Label(toastt, text='Professor: %s'%(pmtmask))#.grid(row=3, column=2, sticky='w', padx=45)
        subf = tkinter.Frame(toast, width= 475, height= 100, bg='light gray')#.pack()#grid(row=4, column=0)
        btconf = tkinter.Button(toastt,text='confirmar', command= lambda:newStd('curso','C')).grid(row=3, column=1, pady=5)
        
        lbID.grid(row=1, column=2, pady=5, padx=45, sticky= 'w')
        lbMAT.grid(row=2 , column=2, pady= 5, padx= 45)
        


        toast.mainloop()
        return

def tstConf(val):
    ''' altera variavel conf para True ou False
        @param val: True ou False
    '''
    global conf

    if val:
        conf= True
    else:
        conf= False

    return conf    

def shwTyp():
    ''' altera variavel choosed 
    '''
    global choosed, sFRSpinbox,val
    choosed = sFRSpinbox.get()

    
    return choosed  

def ftMat(datab,typ):
    '''
    cria um numero de matricula 
    @param datab: table que sera utilizada 
    @param typ: digito inicial da matricula podendo ser S: estudantes ou T: professor ou C: curso 
    '''

    global mmt, nid
    newId(datab)
    sY = date.today().year
    sM = date.today().month
    nnid = str(nid).zfill(6)
    sY = str(sY).zfill(2)
    mmt= str(typ)+str(sY)+str(sM)+nnid    

    return mmt

def newId(datab):
    global shwSFtlist,nid
    opendb()


    nid1 = len(cur.execute('SELECT * from %s' %(datab)).fetchall())
    nid1 = nid1+1
    #print('linhas a serem validadas : %s ' %(nid1))
    for i in range(nid1):
        #print('atual i : %s' %(i))
        cur.execute("SELECT id FROM %s WHERE id LIKE %s" %(datab,i))
        founded = cur.fetchone()
        if founded:
            #print('row %s esta oculpada' %(i))
            a=0
        else:
            #print('row limpa %s' %(i))
            nid = i
            break     
    con.close()
    return nid

def updlblx(datab,typ):
            
        if typ == 'C':
            newId(datab)
            ftMat(datab,typ)
            lbID.grid_forget()
            lbMAT.grid_forget()
            lbID.config(text='ID: %s'%(nid))
            lbMAT.config(text='Matricula: %s'%(mmt))
            lbSprof.config(text='Professor: %s'%(pmtmask))
            lbID.grid(row=1, column=2, pady=5, padx=45, sticky= 'w')
            lbMAT.grid(row=2 , column=2, pady= 5, padx= 45)

        
        else:
            newId(datab)
            ftMat(datab,typ)
            stdLii.grid_forget()
            stdLmmt.grid_forget()
            stdLii.config(text='ID: %s'%(nid))
            stdLmmt.config(text='Matricula: %s'%(mmt))
            stdBxname.delete(0, tkinter.END)
            stdBxid.delete(0, tkinter.END)
            stdBxmtr.delete(0, tkinter.END)
            stdLii.grid(row= 0, column= 2, padx= 50, pady= 2)
            stdLmmt.grid(row=1 , column=2, padx= 50, pady= 2)

def ftEntry(self, typ, event = None):
    '''
        formata a entry

        @param self: box qual sera formatada
        @param typ: 0 caso a formatação seja de cpf, 1 caso a formatação seja de apenas letras, 2 para caso seja apenas numeros
    '''

    text = self.get().replace(".", "").replace("-", "")[:11]
    new_text = ""

    if event.keysym.lower() == "backspace": return
    

    for index in range(len(text)):
        if typ == 0:
            if not text[index] in "0123456789": continue
            if index in [2, 5]: new_text += text[index] + "."
            elif index == 8: new_text += text[index] + "-"
            else: new_text += text[index]
        elif typ == 1:
            if text[index] in "0123456789": continue
            else: new_text += text[index]
        elif typ == 2:
            if not text[index] in "0123456789": continue
            else: new_text += text[index] 

    self.delete(0, "end")
    self.insert(0, new_text)

    return


def newStd(datab, typ):
        ''' cria um novo castro no banco de dados

            @param datab: banco de dados do cadastro
            @param typ: o tipo de cadastro, sendo S: estudantes , P: professor, C: curso 
        '''
        if typ == 'C':
            nm = bxname.get()
            cg = spcg.get()            
            newId(datab)
            ftMat(datab,typ)

            opendb()
            cur.execute("INSERT INTO curso (id, nome, cgHoraria, professor) VALUES (?, ?, ?, ?)", (nid, nm, cg, pmt))
            con.commit()
            cur.execute("SELECT id FROM curso WHERE id = %s" %(nid))
            founded = cur.fetchone()
            con.close()
            if founded:
                updlblx(datab,typ)
                ccSFrame.pack_forget()
                shwList(ccSFrame, ccFrame,'curso',2)                
                tstMensage(0, 'O curso foi cadastrado com sucesso.')

                
            else:
                tstMensage(2, 'Nao foi possivel criar o cadastro do curso. \n Ouve algum erro.')
        
        else:       
            stdname = stdBxname.get()
            stdmtr = stdBxmtr.get()
            stdid = stdBxid.get()
            
            newId(datab)
            ftMat(datab,typ)
            opendb()
            cur.execute("INSERT INTO %s (id, nome, cpf, matricula, senha) VALUES (?, ?, ?, ?, ?)" %(datab), (nid, stdname + " ", stdmtr, mmt ,stdid))
            con.commit()
            cur.execute("SELECT id FROM %s WHERE id IN %s" %(datab,nid))
            founded = cur.fetchone()
            con.close()
            if founded:
                updlblx(datab,typ)
                tstMensage(0, 'O %s foi cadastrado com sucesso' %(datab))
                
            else:
                tstMensage(2, 'Nao foi possivel criar o cadastro do %s. \n Ouve algum erro.' %(datab))

def entryFrame(self,datab,typ):
    ''' abre o Frame de cadastro 
    
        @param self: O frame a qual pertence.
        @param datab: base de dados utilizada
        @param typ: o tipo de cadastro, sendo S: estudantes , P: professor

    ''' 
    global stdBxname, stdBxmtr, stdBxid,nid, stdLmmt, stdLii

    newId(datab)
    ftMat(datab,typ)
    stdFnew = tkinter.Frame(self, width= 725, height= 400, bg= 'white')
    stdFnew.pack(fill='both', expand=True)
    stdLname = tkinter.Label(stdFnew, text="Nome :" , background= "white", font= "Inder 12")
    stdLname.grid(row= 0, column= 0, pady= 2, stick='w')
    stdBxname = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxname.grid(row= 0, column= 1, pady= 2)
    stdBxname.bind("<KeyRelease>", lambda eff: ftEntry(stdBxname, 1, eff))
    stdLmtr = tkinter.Label(stdFnew, text="CPF :" , background= "white", font= "Inder 12")
    stdLmtr.grid(row= 1, column= 0, pady= 2, stick='w')
    stdBxmtr = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxmtr.grid(row= 1, column= 1, pady= 2)
    stdBxmtr.bind("<KeyRelease>", lambda eff: ftEntry(stdBxmtr, 0, eff))
    stdLid = tkinter.Label(stdFnew, text="senha :" , background= "white", font= "Inder 12")
    stdLid.grid(row= 2, column= 0, pady= 2, stick='w')
    stdBxid = tkinter.Entry(stdFnew, width=20, background='white', borderwidth= 3, relief= 'solid')
    stdBxid.grid(row= 2, column= 1, pady= 2)
    stdBxid.bind("<KeyRelease>", lambda eff: ftEntry(stdBxid, 2, eff)) 
    stdGbutton1 = tkinter.Button(stdFnew, text='Cancelar', command= lambda:[stdBxname.delete(0, tkinter.END),stdBxid.delete(0, tkinter.END),stdBxmtr.delete(0, tkinter.END), closeFrame(stdFnew)] )
    stdGbutton1.grid(row= 3, column= 0, stick='w', padx= 10)
    stdGbutton2 = tkinter.Button(stdFnew, text='Confirmar', command= lambda:[newStd(datab,typ)])
    stdGbutton2.grid(row= 3, column= 1, padx= 40)
    stdLii = tkinter.Label(stdFnew, text='ID: %s'%(nid) )
    stdLii.grid(row= 0, column= 2, padx= 50, pady= 2)
    stdLmmt = tkinter.Label(stdFnew, text='Matricula: %s'%(mmt) )
    stdLmmt.grid(row=1 , column=2, padx= 50, pady= 2)
    stdGbutton3 = tkinter.Button(stdFnew, text='Cursos', command= lambda:[])
    stdGbutton3.grid(row = 2, column=2, padx=50, pady = 2)


def updsl():
    
    lbSprof.grid_forget()
    lbSprof.config(text='Professor: %s'%(pmtmask))
    lbSprof.grid(row=3, column=2, sticky='w', padx=45)
    

def SlList(event):
    global curI,subf,pmt, pmtmask,lbSprof
    
    opendb()
    curI = shwSFtlist.focus()
    cur.execute("SELECT id FROM professores WHERE id LIKE ?" ,(curI))
    pmt = cur.fetchone()
    pmt = pmt[0]

    cur.execute("SELECT nome FROM professores WHERE id LIKE ?" ,(curI))
    pmtmask = cur.fetchone()
    pmtmask = pmtmask[0]
    subf.pack_forget()
    updsl()
    openFrame(ccFrame)

    con.close()
    return pmt, pmtmask

def delStd(datab):


    global shwSFtlist
    datab= datab.get()
    opendb()
    #curI = shwSFtlist.focus()
    for i in range(len(shwSFtlist.selection())):
        curI = shwSFtlist.selection()[i]
        #curI = ("'"+curI+"'")
        #print(cur.execute("""select from curso where id = %s""") %(curI))
        cur.execute("DELETE FROM %s WHERE id = %s" %(datab, curI))
    con.commit()
    con.close()




#criar as tabelas ALUNO / PROFESSORES / curso
opendb()
cur.execute(""" 
                CREATE TABLE IF NOT EXISTS estudantes
                ( 
                    id int PRIMARY KEY NOT NULL ,
                    nome VARCHAR(40) NOT NULL, 
                    cpf VARCHAR(15) UNIQUE NOT NULL, 
                    matricula VARCHAR(20) NOT NULL,
                    senha INT NOT NULL
                ) 
            """)
            
cur.execute(""" 
                CREATE TABLE IF NOT EXISTS professores
                ( 
                    id int PRIMARY KEY NOT NULL ,
                    nome VARCHAR(40) NOT NULL, 
                    cpf VARCHAR(15) UNIQUE NOT NULL, 
                    matricula VARCHAR(20) NOT NULL,
                    senha INT NOT NULL
                ) 
            """)

cur.execute(""" 
                CREATE TABLE IF NOT EXISTS curso
                ( 
                    id int PRIMARY KEY NOT NULL,
                    nome VARCHAR(40) NOT NULL,  
                    cgHoraria INT,
                    professor,
                    FOREIGN KEY (professor) REFERENCES professores(id)

                ) 
            """)
            
cur.execute(""" 
                CREATE TABLE IF NOT EXISTS classes
                ( 
            	    ID int not null,
	                cursoID int not null,
	                estudantesId int not null,
            	    turno varchar(10) not null,
                    FOREIGN KEY (cursoID) REFERENCES curso(id),
                    FOREIGN KEY (estudantesID) REFERENCES estudantes(id)

                ) 
            """)                        

con.commit()
con.close()


# criar janela raiz
root = tkinter.Tk()
root.title("Cadastro Institucional ")
root.minsize(width=750, height= 400)

rmenu = tkinter.Frame(root, height=20, width=750, bg='white')
rmenu.pack(fill='x')

mniconH =tkinter.PhotoImage(file = r'assets\\home.png', master=rmenu).subsample(2,2)
rmbtH = tkinter.Button(rmenu, image= mniconH,width=20,height=20, relief='groove',command= lambda:closeFrame(nowFrame)).grid(row=0,column=0)
rmbtAlunos = tkinter.Button(rmenu, text='Alunos',width=6, relief='groove', command= lambda:openFrame(stdFrame)).grid(row=0,column=1)
rmbtProf = tkinter.Button(rmenu, text='Professores',width=9, relief='groove', command= lambda: openFrame(tchFrame)).grid(row=0,column=2)
rmbtcur = tkinter.Button(rmenu, text='Curso',width=5, relief='groove', command= lambda:shwList(ccSFrame,ccFrame,'curso',typ=2)).grid(row=0,column=3)
rmbtexibir = tkinter.Button(rmenu, text='Exibir',width=6, relief='groove', command=lambda:[shwList(shwSFrame,shwFrame,'estudantes')]).grid(row=0,column=4)
rmbtfechar = tkinter.Button(rmenu, text='Fechar',width=6, relief='groove', command= lambda:[con.close(), root.destroy()]).grid(row=0,column=5)

#Frame do estudante
stdFrame = tkinter.Frame(root, height= 400 , width= 750, background='white')
stdSubframe1 = tkinter.Frame(stdFrame, width=25, height= 25, background='Black')
stdSubframe1.pack(fill= 'x', side= 'top', expand= False)
stdSubfLabel = tkinter.Label(stdSubframe1, text='Área dos Alunos', fg='white',bg='black', font=' 14')
stdSubfLabel.pack(side='left')
stdSubframe2 = tkinter.Frame(stdFrame, width=25, height= 25, background='Black')
stdSubframe2.pack(fill= 'x', side= 'top', expand= False)
stdSubf2Button1 = tkinter.Button(stdSubframe2, text='Adicionar', command= lambda:entryFrame(stdFrame,'estudantes','S'), bg='black', fg='white', bd=0)
stdSubf2Button1.pack( side= 'left')
stdSubf2Button2 = tkinter.Button(stdSubframe2, text='excluir', command=lambda:[shwList(choosed)], bg='black', fg='white', bd=0)
stdSubf2Button2.pack( side= 'left')
stdSL1 = tkinter.Label(stdSubframe2,padx=100, bg='black')
stdSL1.pack(side='left')
stdSubf2Entry1 = tkinter.Entry(stdSubframe2, width=25, background='black', relief= 'solid', fg='white', highlightbackground='white', highlightthickness= 1)
stdSubf2Entry1.pack(side='left')
stdSubf2Button3 = tkinter.Button(stdSubframe2, text='Pesquisar', command= lambda:[seachStd(stdSubf2Entry1,0)], bg='black', fg='white', bd=0)
stdSubf2Button3.pack( side= 'left')
nowFrame = stdFrame

# Frame do professor
tchFrame = tkinter.Frame(root, height= 400 , width= 750, background='white')
tchSubframe1 = tkinter.Frame(tchFrame, width=25, height= 25, background='Black')
tchSubframe1.pack(fill= 'x', side= 'top', expand= False)
tchSubfLabel = tkinter.Label(tchSubframe1, text='Área dos Professores', fg='white',bg='black', font=' 14')
tchSubfLabel.pack(side='left')
tchSubframe2 = tkinter.Frame(tchFrame, width=25, height= 25, background='Black')
tchSubframe2.pack(fill= 'x', side= 'top', expand= False)
tchSubf2Button1 = tkinter.Button(tchSubframe2, text='Adicionar', command= lambda:entryFrame(tchFrame,'professores','P'), bg='black', fg='white', bd=0)
tchSubf2Button1.pack( side= 'left')
tchSubf2Button2 = tkinter.Button(tchSubframe2, text='excluir', command=lambda:[shwList(tchSubframe1,tchFrame,choosed)], bg='black', fg='white', bd=0)
tchSubf2Button2.pack( side= 'left')
tchSL1 = tkinter.Label(tchSubframe2,padx=100, bg='black')
tchSL1.pack(side='left')
tchSubf2Entry1 = tkinter.Entry(tchSubframe2, width=25, background='black', relief= 'solid', fg='white', highlightbackground='white', highlightthickness= 1)
tchSubf2Entry1.pack(side='left')
tchSubf2Button3 = tkinter.Button(tchSubframe2, text='Pesquisar', bg='black', fg='white', command= lambda:[seachStd(stdSubf2Entry1,0,0,0)])
tchSubf2Button3.pack( side= 'left')

#Frame de exibição
shwFrame = tkinter.Frame(root)
sFlabel1 = tkinter.Label(shwFrame, text='Consultar.', font='Arial 12 bold')
sFlabel1.grid(column=0, row=0)
shwSFrame = tkinter.Frame(shwFrame,bg='black')
shwSFrame.grid(row= 2, column=1, stick='n')
sFRSpinbox = tkinter.Spinbox(shwSFrame, values= ('estudantes', 'professores'), command= lambda:[shwTyp(),shwList(shwSFrame,shwFrame,choosed)],bg='black',fg='white')
sFRSpinbox.grid(row=1, column=1,sticky='nw',padx=2,pady=4)
shIRF = tkinter.PhotoImage(file = r'assets\\-refresh.png', master=shwSFrame).subsample(2,2)
rfbutton = tkinter.Button(shwSFrame, image= shIRF,bg='black', relief='ridge', width=14,height=14, borderwidth=3, command= lambda: openFrame(shwFrame)).grid(row=1,column=0,padx=2, pady=2)
shwSFrame2 = tkinter.Frame(shwFrame)
shwSFrame2.grid(row=3, column=1,pady=10)
shwSFbutton2 = tkinter.Button(shwSFrame, text='Deletar',command=lambda:[delStd(sFRSpinbox),shwList(shwSFrame,shwFrame,choosed)], width=10, bg='white')#,state='disabled')
shwSFbutton2.grid(row=2, column=2, sticky='en', padx=10)
shwSFEntry1 = tkinter.Entry(shwSFrame2, width=20, relief='solid' )#highlightthickness=2)
shwSFEntry1.grid(row=0, column=1, sticky='se', padx=10)
shwSFbutton4 = tkinter.Spinbox(shwSFrame2, values=('id', 'nome', 'matricula','senha' ),width=12)
shwSFbutton4.grid(row=0, column=0, sticky='se') 
shwSFbutton3 = tkinter.Button(shwSFrame2, text='Pesquisar: ', command=lambda:seachStd(shwSFEntry1, 1, sFRSpinbox, shwSFbutton4))
shwSFbutton3.grid(row=0 , column= 2, sticky='se')


#frame de cadastro de cursos

ccFrame = tkinter.Frame(root)
sccFlabel1 = tkinter.Label(ccFrame, text='Cursos.', font='Arial 12 bold')
sccFlabel1.grid(column=0, row=0)
ccSFrame = tkinter.Frame(ccFrame,bg='black')
ccSFrame.grid(row= 2, column=1, stick='n')
ccFRSpinbox = tkinter.Spinbox(ccSFrame, values= ('curso'), command= lambda:[shwTyp(),shwList(ccSFrame,ccFrame, 'curso',2)],bg='black',fg='white')
ccFRSpinbox.grid(row=1, column=1,sticky='nw', pady=4, padx=2)
ccIRF = tkinter.PhotoImage(file = r'assets\\-refresh.png', master=ccSFrame).subsample(2,2)
rfbutton = tkinter.Button(ccSFrame, image= ccIRF,bg='black', relief='ridge', width=14,height=14,borderwidth=3, command= lambda: openFrame(ccFrame)).grid(row=1,column=0,padx=2, pady=2)
ccSFrame2 = tkinter.Frame(ccFrame)
ccSFrame2.grid(row=3, column=1,pady=10)
ccSFbutton2 = tkinter.Button(ccSFrame, text='Deletar',command=lambda:[delStd(ccFRSpinbox),shwList(ccSFrame, ccFrame,'curso',2)], width=10, bg='white')#,state='disabled')
ccSFbutton2.grid(row=2, column=2, sticky='en', padx=10)
ccSFbutton4 = tkinter.Button(ccSFrame, text='Novo.', command=lambda:tstMensage(3), padx=20)
ccSFbutton4.grid(row= 2, column= 2, sticky='n', padx=10, pady= 35)


ccSFEntry1 = tkinter.Entry(ccSFrame2, width=20, relief='solid' )#highlightthickness=2)
ccSFEntry1.grid(row=0, column=1, sticky='se', padx=10)
ccSFbutton4 = tkinter.Spinbox(ccSFrame2, values=('id', 'nome', 'matricula','senha' ),width=12)
ccSFbutton4.grid(row=0, column=0, sticky='se') 
ccSFbutton3 = tkinter.Button(ccSFrame2, text='Pesquisar: ', command=lambda:seachStd(ccSFEntry1, 1, ccFRSpinbox, ccSFbutton4))
ccSFbutton3.grid(row=0 , column= 2, sticky='se')

root.mainloop()